## Configuration and authentication demo

This notebook demonstrates how to generate your config file. The most complicated part of this is generating a token that can be passed to the ADAM REST server for access to API methods.

In [1]:
import sys
from os.path import expanduser

# default adam location - alter directory structure if you decide to use a different path
# for example: adam_home_defined = expanduser("~") + "your_project_name/scenario1/adam_home"
adam_home_defined = expanduser("~") + "/adam_home" # default location

# Add adam path to system in order to import adam
sys.path.append(adam_home_defined)
# import adam and adam modules required
import adam
from adam import adam_config
from adam import Auth
from adam import ConfigManager
from adam import RestRequests

import webbrowser
import urllib
import os
from shutil import copyfile

In [2]:
# Set up adam paths
ADAMpaths = adam_config.setPaths.initPaths(adam_home_defined)

Changing adam home path to =  C:\Users\macuser/adam_home


Attempts to read config that is already set up from a config.json file in your working directory. Feel free to point it somewhere else. If it fails to read that file, it will copy the template from adam.

In [3]:
# Print paths from adam_config.py
print("adam_path = ", ADAMpaths[0])
print("data_path = ", ADAMpaths[1])
print("env_template_path = ", ADAMpaths[2])
print("env_config_path = ", ADAMpaths[3])
print("ephem_path = ", ADAMpaths[4])
print("MY_functions_path = ", ADAMpaths[5])

adam_path =  C:\Users\macuser/adam_home/
data_path =  C:\Users\macuser/adam_home/data
env_template_path =  C:\Users\macuser/adam_home/config/adam_config_template.json
env_config_path =  C:\Users\macuser/adam_home/config/adam_config.json
ephem_path =  C:\Users\macuser/adam_home/data/ephem
MY_functions_path =  C:\Users\macuser/adam_home/MY_functions


In [4]:
#Attempts to read config that is already set up from an adam_config.json file in adam_home/config directory.
#Feel free to point it somewhere else. If it fails to read that file, it will copy the template from adam.
config_file  = ADAMpaths[3]
template_file = ADAMpaths[2]

try:
    f = open(config_file)
    f.close()
except:
    copyfile(template_file, config_file)

config_manager = ConfigManager(config_file)
config = config_manager.get_config()

Now attempt to authenticate with the token in the config. If that fails, it tries to reauthenticate. This is a flow that directs a user to a page where they can log in and get a token. If that new token works, this will update the config file so the token can be used next time.

In [5]:
url = config.get_url()
rest = RestRequests(url)
auth = Auth(rest)

In [6]:
token = config.get_token()

try:
    auth.authenticate(token)
    if auth.get_logged_in():
        print('Welcome, ' + auth.get_user())
    else:
        input("A website will pop up and you will be given options to log in to "
              "retrieve a token. Please copy the token and return here. Press enter to show popup and continue...")
        o = urllib.parse.urlparse(url)
        token_url = "%s://%s/%s" % (o.scheme, o.netloc, 'token.html')  # Yes, seriously. This is how it's done.
        webbrowser.open(token_url)
        
        token = input("Token: ")
        print('\n')
        if auth.authenticate(token):
            config.set_token(auth.get_token())
            config_manager.to_file(config_file)
            print('Updated config file with token for ' + url)
            print('Welcome, ' + auth.get_user())
        else:
            print('Could not authenticate user.')
except RuntimeError as e:
    print('Encountered server error while attempting to authenticate: ' + str(e))

Welcome, emmie.king@gmail.com


If authentication was successful, the token can now be used for any API method requiring authentication.

Now an easy extra part. Do you have a workspace project? Enter its uuid here. It will be used as a parent of any temporary working projects created.

In [ ]:
# If you need some help finding the uuid, try the following (it might take a few seconds):
# from adam import Projects
# from adam import AuthenticatingRestProxy
# Projects(AuthenticatingRestProxy(RestRequests(config.get_url()), token)).print_projects()

# workspace = input('Workspace project UUID: ')
# config.set_workspace(workspace)
# config_manager.to_file(config_file)